<a href="https://colab.research.google.com/github/U-CovUni/NLP-CourseWork1/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, Embedding, GlobalMaxPool1D, Bidirectional
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/semEval.csv')

# Converting the stance labels to numerical format
stance_mapping = {'AGAINST': 0, 'FAVOR': 1, 'NONE': 2}
data['Stance'] = data['Stance'].map(stance_mapping)

# Splitting the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Initializing the CountVectorizer (Bag of Words)
vectorizer = CountVectorizer(max_features=10000, stop_words='english')

# Fitting and transforming the training data, transform the test data
X_train = vectorizer.fit_transform(train_data['Tweet']).toarray()
X_test = vectorizer.transform(test_data['Tweet']).toarray()

# Extracting the labels
y_train = train_data['Stance']
y_test = test_data['Stance']

# Encoding the labels
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Converting labels to categorical (one-hot encoding)
y_train_categorical = to_categorical(y_train_encoded, num_classes=3)
y_test_categorical = to_categorical(y_test_encoded, num_classes=3)

# Fitting Convolutional Neural Network (CNN)
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(128, 5, activation='relu', input_shape=(input_shape, 1)),
        MaxPooling1D(pool_size=2),
        Dropout(0.5),
        Conv1D(128, 5, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.5),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model
# Train and evaluate models
input_shape = X_train.shape[1]

# Reshape data for CNN
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


cnn_model = create_cnn_model(input_shape)
print("\nTraining Convolutional Neural Network...")
cnn_history = cnn_model.fit(X_train_cnn, y_train_categorical, epochs=10, validation_data=(X_test_cnn, y_test_categorical), batch_size=32)
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test_categorical, verbose=2)
print(f'CNN Test Accuracy: {cnn_accuracy}')

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training Convolutional Neural Network...
Epoch 1/10
30/71 ━━━━━━━━━━━━━━━━━━━━ 3:23 5s/step - accuracy: 0.4568 - loss: 1.0954